In [2]:
#pip install sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 625.7/625.7 kB 3.9 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from sqlalchemy import create_engine

# Crear una conexión a una base de datos SQLite
engine = create_engine('sqlite:///ejemplo_alchemy.db', echo=True)

In [2]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String

Base = declarative_base()

class Usuario(Base):
    __tablename__ = 'usuarios'

    id = Column(Integer, primary_key=True)
    nombre = Column(String)
    email = Column(String)

    def __repr__(self):
        return f"<Usuario(nombre='{self.nombre}', email='{self.email}')>"

# Crear las tablas en la base de datos
Base.metadata.create_all(engine)

2024-08-08 13:38:35,483 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-08 13:38:35,484 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("usuarios")
2024-08-08 13:38:35,484 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-08 13:38:35,485 INFO sqlalchemy.engine.Engine COMMIT


In [3]:
from sqlalchemy.orm import sessionmaker
#Las sesiones se utilizan para manejar las transacciones de la base de datos:
Session = sessionmaker(bind=engine)
session = Session()

In [4]:
nuevo_usuario = Usuario(nombre='Juan', email='juan@ejemplo.com')
session.add(nuevo_usuario)
session.commit()

2024-08-08 13:40:28,510 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-08 13:40:28,517 INFO sqlalchemy.engine.Engine INSERT INTO usuarios (nombre, email) VALUES (?, ?)
2024-08-08 13:40:28,519 INFO sqlalchemy.engine.Engine [generated in 0.00263s] ('Juan', 'juan@ejemplo.com')
2024-08-08 13:40:28,523 INFO sqlalchemy.engine.Engine COMMIT


In [5]:

usuarios = session.query(Usuario).all()
for usuario in usuarios:
    print(usuario)

juan = session.query(Usuario).filter_by(nombre='Juan').first()
print(juan)

2024-08-08 13:45:00,617 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-08 13:45:00,620 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nombre AS usuarios_nombre, usuarios.email AS usuarios_email 
FROM usuarios
2024-08-08 13:45:00,620 INFO sqlalchemy.engine.Engine [generated in 0.00054s] ()
<Usuario(nombre='María', email='maria@ejemplo.com')>
<Usuario(nombre='Juan', email='juan@ejemplo.com')>
2024-08-08 13:45:00,623 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nombre AS usuarios_nombre, usuarios.email AS usuarios_email 
FROM usuarios 
WHERE usuarios.nombre = ?
 LIMIT ? OFFSET ?
2024-08-08 13:45:00,623 INFO sqlalchemy.engine.Engine [generated in 0.00050s] ('Juan', 1, 0)
<Usuario(nombre='Juan', email='juan@ejemplo.com')>


In [6]:
#ACTUALIZACIÓN DE DATOS

juan.email = 'juan_nuevo@ejemplo.com'
session.commit()

2024-08-08 13:46:11,200 INFO sqlalchemy.engine.Engine UPDATE usuarios SET email=? WHERE usuarios.id = ?
2024-08-08 13:46:11,201 INFO sqlalchemy.engine.Engine [generated in 0.00095s] ('juan_nuevo@ejemplo.com', 2)
2024-08-08 13:46:11,201 INFO sqlalchemy.engine.Engine COMMIT


In [8]:
juan = session.query(Usuario).filter_by(nombre='Juan').first()
print(juan)

2024-08-08 13:46:52,072 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nombre AS usuarios_nombre, usuarios.email AS usuarios_email 
FROM usuarios 
WHERE usuarios.nombre = ?
 LIMIT ? OFFSET ?
2024-08-08 13:46:52,074 INFO sqlalchemy.engine.Engine [cached since 111.5s ago] ('Juan', 1, 0)
<Usuario(nombre='Juan', email='juan_nuevo@ejemplo.com')>


In [9]:
# Eliminar Datos
session.delete(juan)
session.commit()

2024-08-08 13:49:42,652 INFO sqlalchemy.engine.Engine DELETE FROM usuarios WHERE usuarios.id = ?
2024-08-08 13:49:42,652 INFO sqlalchemy.engine.Engine [generated in 0.00078s] (2,)
2024-08-08 13:49:42,655 INFO sqlalchemy.engine.Engine COMMIT


## RELACIONES

In [10]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Post(Base):
    __tablename__ = 'posts'

    id = Column(Integer, primary_key=True)
    titulo = Column(String)
    contenido = Column(String)
    usuario_id = Column(Integer, ForeignKey('usuarios.id'))    
    usuario = relationship("Usuario", back_populates="posts")


2024-08-08 09:55:41,346 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-08 09:55:41,348 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("usuarios")
2024-08-08 09:55:41,349 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-08 09:55:41,350 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("posts")
2024-08-08 09:55:41,350 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-08 09:55:41,351 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("posts")
2024-08-08 09:55:41,351 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-08 09:55:41,353 INFO sqlalchemy.engine.Engine 
CREATE TABLE posts (
	id INTEGER NOT NULL, 
	titulo VARCHAR, 
	contenido VARCHAR, 
	usuario_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(usuario_id) REFERENCES usuarios (id)
)


2024-08-08 09:55:41,354 INFO sqlalchemy.engine.Engine [no key 0.00049s] ()
2024-08-08 09:55:41,496 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:

Usuario.posts = relationship("Post", order_by=Post.id, back_populates="usuario")

Base.metadata.create_all(engine)

In [11]:
usuario = Usuario(nombre='María', email='maria@ejemplo.com')
post = Post(titulo='Mi primer post', contenido='Hola mundo!', usuario=usuario)

session.add(usuario)
session.add(post)
session.commit()

# Consultar los posts de un usuario
maria = session.query(Usuario).filter_by(nombre='María').first()
for post in maria.posts:
    print(f"{maria.nombre}: {post.titulo}")

2024-08-08 09:55:56,485 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-08 09:55:56,488 INFO sqlalchemy.engine.Engine INSERT INTO usuarios (nombre, email) VALUES (?, ?)
2024-08-08 09:55:56,488 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ('María', 'maria@ejemplo.com')
2024-08-08 09:55:56,492 INFO sqlalchemy.engine.Engine INSERT INTO posts (titulo, contenido, usuario_id) VALUES (?, ?, ?)
2024-08-08 09:55:56,494 INFO sqlalchemy.engine.Engine [generated in 0.00167s] ('Mi primer post', 'Hola mundo!', 1)
2024-08-08 09:55:56,498 INFO sqlalchemy.engine.Engine COMMIT
2024-08-08 09:55:56,638 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-08 09:55:56,644 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nombre AS usuarios_nombre, usuarios.email AS usuarios_email 
FROM usuarios 
WHERE usuarios.nombre = ?
 LIMIT ? OFFSET ?
2024-08-08 09:55:56,646 INFO sqlalchemy.engine.Engine [cached since 109.1s ago] ('María', 1, 0)
2024-08-08 09:55:56,656 INFO sqla

RELACIONES UNO A UNO

In [12]:
from sqlalchemy import Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship, declarative_base

Base = declarative_base()

class Usuario(Base):
    __tablename__ = 'usuarios'
    id = Column(Integer, primary_key=True)
    nombre = Column(String)
    perfil = relationship("Perfil", back_populates="usuario", uselist=False)

class Perfil(Base):
    __tablename__ = 'perfiles'
    id = Column(Integer, primary_key=True)
    biografia = Column(String)
    usuario_id = Column(Integer, ForeignKey('usuarios.id'))
    usuario = relationship("Usuario", back_populates="perfil")

RELACIONES UNO A MUCHOS

In [13]:
class Usuario(Base):
    __tablename__ = 'usuarios'
    id = Column(Integer, primary_key=True)
    nombre = Column(String)
    posts = relationship("Post", back_populates="autor")

class Post(Base):
    __tablename__ = 'posts'
    id = Column(Integer, primary_key=True)
    titulo = Column(String)
    usuario_id = Column(Integer, ForeignKey('usuarios.id'))
    autor = relationship("Usuario", back_populates="posts")

/tmp/ipykernel_17037/339745855.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Usuario, and will be replaced in the string-lookup table.
  class Usuario(Base):


InvalidRequestError: Table 'usuarios' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.